In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from scipy.stats import multivariate_normal as mvn
from scipy.stats import t
import seaborn as sns
from interface import Sampler, to_numpy, writeChains, loadChains

cols = np.array(sns.color_palette(as_cmap=True))

In [ ]:
ndata = 150
ntest = 150

In [ ]:
fixed_covs_train = np.vstack(
    [np.random.normal(size=(ndata, 2)) + np.array([-3, 0]),
     np.random.normal(size=(ndata, 2)) + np.array([3, 0])])
fixed_covs_test = np.vstack(
    [np.random.normal(size=(ntest, 2)) + np.array([-3, 0]),
     np.random.normal(size=(ntest, 2)) + np.array([3, 0])])
true_clus_train = np.concatenate([
    np.zeros(ndata), np.ones(ndata)
]).astype(int)
true_clus_test = np.concatenate([
    np.zeros(ntest), np.ones(ntest)
]).astype(int)

In [ ]:
plt.scatter(fixed_covs_train[:, 0], fixed_covs_train[:, 1], c=cols[true_clus_train])
plt.show()

In [ ]:
import scipy

fdim = 2
ldim = 5
rdim = 3

phi0 = [
    np.eye(3) * 1.1,
    np.array([[1.1, -0.1, 0], [-0.1, 1.1, 0], [0, 0, 0.9]]) 
]
sigma0 = np.eye(3) * 0.2
beta0 = np.zeros((rdim, ldim))
gamma0 =np.zeros((rdim, fdim))

ndata = len(true_clus_train)


def generate_data(fixed_cov, clus, T=10):
    long_cov = np.random.normal(size=(T, ldim))
    y = np.zeros((T, rdim))
    y[0, :] = np.random.normal(loc=5, scale=0.7, size=rdim)
    for t in range(1, T):
        mean = np.matmul(phi0[clus], y[t-1, :]) + \
              np.matmul(beta0, long_cov[t, :]) + \
              np.matmul(gamma0, fixed_cov) 
        err = scipy.stats.t.rvs(2, scale=0.2, size=rdim)
        y[t, :] = mean + err    
    return y, long_cov


T = 10

resps = []
long_covs = []
insample_test_resp = []
insample_test_long = []

for i in range(ndata):
    y, long = generate_data(
        fixed_covs_train[i, :], true_clus_train[i], T)
    if (i >= 100 and i < 150) or (i > 250) :
        insample_test_resp.append(y[5:, :])
        insample_test_long.append(long[5:, :])
        y = y[:5, :]
        long = long[:5, :]
    resps.append(y)
    long_covs.append(long)

    

test_y = []
test_long = []

for i in range(ntest):
    c = np.random.choice(np.arange(2))
    y, long = generate_data(
        fixed_covs_test[i, :], true_clus_test[i], T)
    test_y.append(y)
    test_long.append(long)
    true_clus_test[i] = c

In [ ]:
fig, axis = plt.subplots(nrows=1, ncols=3, figsize=(10, 3))
for i in range(ndata):
    for j in range(rdim):
        axis[j].plot(np.arange(resps[i].shape[0]), resps[i][:, j], color=cols[true_clus_train[i]])

In [ ]:
lsb_sampler = Sampler(50, "LSB")
lsb_sampler.set_prior(
    phi00=np.eye(rdim),
    v00=np.eye(rdim * rdim) * 0.15,
    nu=30,
    tau=15,
    lamb=1.0,
    sigma0=np.eye(rdim) * 0.4,
    beta0=np.zeros((rdim, ldim)),
    gamma0=np.zeros((rdim, fdim)),
    alpha0=np.zeros(fdim),
    vara=10,
)
is_missing = []
lsb_chains = lsb_sampler.run_mcmc(
    0, 10000, 10000, 10, resps, long_covs, fixed_covs_train, is_missing)

In [ ]:
dp_sampler = Sampler(50, "DP")
dp_sampler.set_prior(
    phi00=np.zeros((rdim, rdim)),
    v00=np.eye(rdim * rdim),
    nu=30,
    tau=15,
    lamb=0.1,
    sigma0=np.eye(rdim) * 0.4,
    beta0=np.zeros((rdim, ldim)),
    gamma0=np.zeros((rdim, fdim)),
    alpha0=np.zeros(fdim),
)
is_missing = []
dp_chains = dp_sampler.run_mcmc(
    0, 10000, 10000, 10, resps, long_covs, fixed_covs_train, is_missing)

In [ ]:
def score(samples, true_y):
    pred_mean = np.median(samples, axis=0).reshape(*true_y.shape)
    return np.sum((pred_mean - true_y) ** 2) / true_y.shape[0]


def get_out_of_sample_mse(sampler, test_long, test_fix, test_y):
    mse_full = []
    ntest = len(y)
    for i in range(ntest):
        pred_full = sampler.sample_predictive(
            test_long[i], test_fix[i], test_y[i][0, :])
        mse_full.append(score(pred_full, test_y[i]))
    return mse_full


def get_in_sample_mse(sampler, test_y, test_long, data_idx):
    mse_insample = []
    for i in range(100):
        pred_insample = dp_sampler.predict_insample(
            data_idx, test_long[i], fixed_covs[data_idx, :], 5)
        mse_insample_dp.append(score(pred_insample, test_y[i]))
    return mse_insample

In [ ]:
oos_mse_lsb = get_out_of_sample_mse(
    lsb_sampler, test_long, fixed_covs_test, test_y)

oos_mse_dp = get_out_of_sample_mse(
    dp_sampler, test_long, fixed_covs_test, test_y)

In [ ]:
print(np.median(oos_mse_dp), np.mean(oos_mse_dp), np.std(oos_mse_dp))

In [ ]:
print(np.median(oos_mse_lsb), np.mean(oos_mse_lsb), np.std(oos_mse_lsb))

In [ ]:
from sklearn.metrics import adjusted_rand_score

In [ ]:
aris = []
for c in dp_chains:
    aris.append(adjusted_rand_score(c.clus_allocs, true_clus_train))
np.mean(aris)

In [ ]:
aris = []
for c in lsb_chains:
    aris.append(adjusted_rand_score(c.clus_allocs, true_clus_train))
np.mean(aris)

In [ ]:
def get_in_sample_mse(sampler):
    out = []
    curr_idx = 0
    for i in range(ndata):
        if (i >= 100 and i < 150) or (i > 250):
            pred_insample = sampler.predict_insample(
                i, insample_test_long[curr_idx], fixed_covs_train[i, :], 5)
            out.append(score(pred_insample, insample_test_resp[curr_idx]))
            curr_idx += 1
    return out

In [ ]:
insample_mse_dp = get_in_sample_mse(dp_sampler)

insample_mse_ldp = get_in_sample_mse(lsb_sampler)

print(np.median(insample_mse_dp), np.mean(insample_mse_dp), np.std(insample_mse_dp))

print(np.median(insample_mse_ldp), np.mean(insample_mse_ldp), np.std(insample_mse_ldp))